## ViCTOR TS onsets

1. check for reverse coding and multiple responses - fix manually in excel, then generate new accuracies. I counted the last response given, changed no response to 0.

For reverse coding: `=IF(CELL=1,4,IF(CELL=3,2,IF(CELL=4,1,IF(CELL=2,3,0))))`

For multiple responses: `=VALUE(IF(ISBLANK(CELL), 0, IF(LEN(CELL>1),RIGHT(CELL,1),CELL)))`

2. Get onset time - copy from the template. TS should start at 30, then add 1.5 and ntimes. And add 30 second every 30 trials (in miliseconds).

3. After preparing the data, run the code below. You need python installed, along with the packages (pandas and xlrd; use e.g., pip install pandas). Note: If the participant made no errors, an empty file will be named 0_0, rename it appropriately (num_Err.txt)


In [3]:
import xlrd
import itertools
import pandas as pd

def orig_order_set(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

book = xlrd.open_workbook('Merge_TS_template.xlsx')
sheet = book.sheet_by_name('Merge_TS')

subs = sheet.col_values(1)[1:]
acc = sheet.col_values(64)[1:]
switch = sheet.col_values(67)[1:]
ons = sheet.col_values(69)[1:]

sub_list = subset = orig_order_set(subs)

data = pd.DataFrame(list(zip(subs, acc, switch, ons)))
data.columns = ['sub', 'acc', 'switch', 'onset']
#data = data[data.switch != 9]
data['display'] = '1.5'
data['weight'] = '1'

data["cond"] = ""
data.loc[(data.acc == 0) | (data.switch == 9), "cond"] = "Err"
data.loc[(data.acc == 1) & (data.switch == 0), "cond"] = "Single"
data.loc[(data.acc == 1) & (data.switch == 1), "cond"] = "DualNS"
data.loc[(data.acc == 1) & (data.switch == 2), "cond"] = "DualSW"

Err = data[(data.acc == 0) | (data.switch == 9)]
Single = data[(data.acc == 1) & (data.switch == 0)]
DualNS = data[(data.acc == 1) & (data.switch == 1)]
DualSW = data[(data.acc == 1) & (data.switch == 2)]

cond_list = [Err, Single, DualNS, DualSW]
by_sub = [j[j['sub'] == i] for i in sub_list for j in cond_list]
columns_needed = ["onset", "display", "weight"]

for i in by_sub:
    i.to_csv(str(int(i.iloc[0,0])) + "_" + str(i.iloc[0,6]) + ".txt", header=False, columns=columns_needed, index=False, sep="\t")

## BETTER TS onsets

In [2]:
import xlrd
import itertools
import pandas as pd

def orig_order_set(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

book = xlrd.open_workbook('C:\\Users\\pauli\\Downloads\\ForPaulina\\BETTER_TS\\MergeV2.xlsx')
sheet = book.sheet_by_name('MergeV2')

# To easily find column the numbers: Excel> File> Options> Formulas> Change reference style to R1C1
# Just remember to subtract 1.
subs = sheet.col_values(1)[1:]
acc = sheet.col_values(65)[1:]
comp = sheet.col_values(70)[1:]
ons = sheet.col_values(72)[1:]
sub_list = subset = orig_order_set(subs)

data = pd.DataFrame(list(zip(subs, acc, comp, ons)))
data.columns = ['sub', 'acc', 'comp', 'onset']
data['display'] = '1.5'
data['weight'] = '1'

data["cond"] = ""
data.loc[(data.acc == 0) | (data.comp == 9), "cond"] = "Err"
data.loc[(data.acc == 1) & (data.comp == 0), "cond"] = "Single"
data.loc[(data.acc == 1) & (data.comp == 1), "cond"] = "NS_comp"
data.loc[(data.acc == 1) & (data.comp == 2), "cond"] = "NS_incomp"
data.loc[(data.acc == 1) & (data.comp == 3), "cond"] = "SW_comp"
data.loc[(data.acc == 1) & (data.comp == 4), "cond"] = "SW_incomp"

Err = data[(data.acc == 0) | (data.comp == 9)]
Single = data[(data.acc == 1) & (data.comp == 0)]
NS_comp = data[(data.acc == 1) & (data.comp == 1)]
NS_incomp = data[(data.acc == 1) & (data.comp == 2)]
SW_comp = data[(data.acc == 1) & (data.comp == 3)]
SW_incomp = data[(data.acc == 1) & (data.comp == 4)]

cond_list = [Err, Single, NS_comp, NS_incomp, SW_comp, SW_incomp]
by_sub = [j[j['sub'] == i] for i in sub_list for j in cond_list]
columns_needed = ["onset", "display", "weight"]

for i in by_sub:
    i.to_csv(str(int(i.iloc[0,0])) + "_" + str(i.iloc[0,6]) + "_V2.txt", header=False, columns=columns_needed, index=False, sep="\t")

## EM onsets 

1. Stim2.RESP and Stim3.RESP need to be combined. I counted the last answer. If no response for S2, take S3; if S2 and S3 given, take last S3; if S2 response only, take last S2; otherwise 0. 
`=IF(LEN(O2)<1,R2,IF(AND(LEN(O2)=1, LEN(R2)>=1), RIGHT(R2,1), IF(LEN(O2)>=1, RIGHT(O2,1), 0)))`

2. Fix accuracies using new responses ` =IF(Corr.ans=Resp.fix,1,0)`

3. Categorize the responses depending on whether the trial was a Target, a Lure, or a Foil, and whether the answer was correct or not.
`=IF(ObjectType="T",IF(Acc.fix=1, "REM", "FRGT"), IF(ObjectType="L", IF(Acc.fix=1, "CR", "FA"), "BLANK"))`

In [1]:
import xlrd
import itertools
import pandas as pd

workbook = xlrd.open_workbook('Merged0730.xlsx')
worksheet = workbook.sheet_by_name('Sheet1')

def divide_chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

def orig_order_set(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

sub = worksheet.col_values(7)[1:]
subset = orig_order_set(sub)
sub_enc = [list(itertools.repeat(i, 128)) for i in subset]
sub_enc = [item for sublist in sub_enc for item in sublist]

#stim_enc = worksheet.col_values(3)[1:129]
stim_enc = ["096","127","069","066","169","005","082","041","180","134",
            "146","108","192","018","140","030","061","086","152","002",
            "021","013","168","040","166","177","068","143","138","036",
            "015","028","044","029","048","060","183","155","073","010",
            "099","190","090","022","072","049","008","131","117","113",
            "055","111","027","035","098","175","070","053","058","123",
            "085","017","120","023","122","182","189","084","149","042",
            "114","172","144","119","051","150","185","025","178","170",
            "012","091","083","067","095","087","191","043","080","109",
            "124","137","129","133","112","103","094","065","132","148",
            "007","050","163","141","009","161","079","016","088","076",
            "037","142","116","089","139","184","032","047","167","006",
            "081","100","174","173","156","145","125","045"]
#stim_ret = worksheet.col_values(12)[1:193]
stim_ret = ["174","113","016","099","152","053","041","159","172","123",
            "163","051","050","160","105","060","034","076","007","049",
            "125","098","085","038","079","191","121","141","177","145",
            "109","161","059","033","017","036","081","169","176","071",
            "140","035","186","069","064","086","162","180","115","181",
            "047","129","006","028","090","063","150","065","073","132",
            "075","102","005","082","156","062","119","178","143","093",
            "021","091","111","158","106","025","103","012","104","084",
            "061","137","185","146","022","133","153","167","144","154",
            "097","187","151","020","039","157","018","095","013","037",
            "008","189","100","149","014","184","088","165","116","175",
            "057","055","080","046","164","077","015","030","166","114",
            "148","124","002","042","010","131","182","168","108","078",
            "027","092","054","074","130","134","138","024","043","026",
            "112","001","147","070","118","094","122","190","048","040",
            "058","056","004","192","171","072","173","044","101","126",
            "142","188","009","083","089","179","096","087","068","110",
            "170","183","067","135","136","117","045","003","032","011",
            "029","031","127","052","066","019","023","155","128","107",
            "120","139"]
rts = stim_ret*len(subset)

onset = worksheet.col_values(4)[1:129]
onsets = onset*len(subset) 

dprime = worksheet.col_values(18)[1:]
dps = list(divide_chunks(dprime, 192)) 

dict_ret = [dict(zip(stim_ret, dps[i])) for i in range(len(dps))]
rec = {k : v1 for k,v1 in zip(subset, dict_ret)}
res = [rec[j][i] for j in rec for i in stim_enc if i in rec[j]]
res_s = list(divide_chunks(res, 128))
dict_enc = dict(zip(subset, res_s))

df = pd.DataFrame(list(zip(sub_enc, res, onsets)))
df.columns = ['sub', 'cond', "onset"]
df['display'] = '4'
df['weight'] = '1'

REM = df[df['cond'] == "REM"]
MISS = df[df['cond'] == "FRGT"]
CR = df[df['cond'] == "CR"]
FA = df[df['cond'] == "FA"]

cond_list = [REM, MISS, CR, FA]
by_sub = [j[j['sub'] == i].sort_values('onset') for i in subset for j in cond_list]
columns_needed = ["onset", "display", "weight"]

for i in by_sub:
    i.to_csv(str(int(i.iloc[0,0])) + "_" + str(i.iloc[0,1]) + ".txt", header=False, columns=columns_needed, index=False, sep="\t")

### Alternative TS onsets - buggy

In [ ]:
# Slow. Theres's a bug that requires the "_None.txt" files to be generated, you can delete these.
import os
from openpyxl import load_workbook

wb = load_workbook('Merge_TS_edit.xlsx')
sheet = wb['Merge_TS']

subs = [sheet['B'+str(i)].value for i in range(2,sheet.max_row+1)]
subs = sorted(list(set(subs)))

# 'B' - subject ID, 'BM' - fixed acc, 'BP' - task condition, 'BR' - onset time

for s in range(len(subs)):
 for i in range(2,sheet.max_row+1):
     if sheet['BM'+str(i)].value == 0 and sheet['B'+str(i)].value == subs[s]:
         file = open(str(subs[s]) + "_Err.txt", "a")
     elif sheet['BM'+str(i)].value == 1 and sheet['BP'+str(i)].value == 0 and sheet['B'+str(i)].value == subs[s]:
         file = open(str(subs[s]) + "_Single.txt", "a")
     elif sheet['BM'+str(i)].value == 1 and sheet['BP'+str(i)].value == 1 and sheet['B'+str(i)].value == subs[s]:
         file = open(str(subs[s]) + "_DualNS.txt", "a")
     elif sheet['BM'+str(i)].value == 1 and sheet['BP'+str(i)].value == 2 and sheet['B'+str(i)].value == subs[s]:
         file = open(str(subs[s]) + "_DualSw.txt", "a")
     else:
         file = open(str(subs[s]) + "_None.txt", "a")
                   
     file.write(str(sheet['BR'+str(i)].value) + "\t" + str(1.5) + "\t" + str(1) + "\n")
file.close()
